In [2]:
import numpy as np
import pandas as pd
from scapy.all import *
import random, glob, os, sys

import tensorflow as tf
import keras

/Users/michaelwrana/opt/anaconda3/lib/python3.9/site-packages/scapy/layers/ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/Users/michaelwrana/opt/anaconda3/lib/python3.9/site-packages/scapy/layers/ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,
2023-02-22 12:10:40.568000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
folder = "/Users/michaelwrana/Desktop/data/"

x = []
y = []
for filename in glob.iglob(folder + '**', recursive=True):
    if os.path.isfile(filename):
        
        # figure out what the label should be based on directory location
        # e.g. data/file1/website1 means file 1 downloaded from website 1
        file_path = filename.split('/')
        site_id = int(file_path[-2][-1])
        file_id = int(file_path[-3][-1])
        
        if(site_id == 1):
            continue
        
        print(site_id)
        print("File: " + str(file_id))
        # load packets from file
        packets = rdpcap(filename)
        
        # configure sequences
        seq_per_file = 100
        seq_len = 5
        packet_sequences = [0] * seq_per_file
        
        # select sequences from packets randomly
        for i in range(seq_per_file):
            start = random.randint(0,len(packets)-seq_len)
            packet_sequence = [raw(packets[j]) for j in range(start, start + seq_len)]
            flat_sequences = [item for sub_list in packet_sequence for item in sub_list]
            packet_sequences[i] = flat_sequences
        
        # assign class labels (based on website or file ID)
        file_y = [file_id] * seq_per_file
        # file_y = [site_id] * seq_per_file
        
        # append to final dataset
        x = x + packet_sequences
        y = y + file_y
        
        
# pad sequences and convert to numpy array
x = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=1000)
x = np.asarray(x)
y = np.asarray(y)

2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2
2
File: 2


In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

2023-02-22 12:16:32.545600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(x, y, epochs=15)

Epoch 1/15
625/625 [==============================] - 1s 1ms/step - loss: 0.5816 - accuracy: 0.6613
Epoch 2/15
625/625 [==============================] - 1s 1ms/step - loss: 0.5747 - accuracy: 0.6784
Epoch 3/15
625/625 [==============================] - 1s 1ms/step - loss: 0.5732 - accuracy: 0.6775
Epoch 4/15
625/625 [==============================] - 1s 1ms/step - loss: 0.5775 - accuracy: 0.6753
Epoch 5/15
625/625 [==============================] - 1s 991us/step - loss: 0.5722 - accuracy: 0.6812
Epoch 6/15
625/625 [==============================] - 1s 1ms/step - loss: 0.5645 - accuracy: 0.6896
Epoch 7/15
625/625 [==============================] - 1s 1ms/step - loss: 0.5618 - accuracy: 0.6865
Epoch 8/15
625/625 [==============================] - 1s 1ms/step - loss: 0.5542 - accuracy: 0.6964
Epoch 9/15
625/625 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.6927
Epoch 10/15
625/625 [==============================] - 1s 1ms/step - loss: 0.5453 - accuracy: 0.70